# Modelos Lineares


In [1]:
# @title Lendo a planilha

import numpy as np
import pandas as pd

df = pd.read_csv("../export/dados.csv", index_col=0)

ciclos = df.index.values
t_feed = df["t_feed"].values
t_rinse = df["t_rinse"].values
t_blow = df["t_blow"].values
t_purge = df["t_purge"].values

purity_H2 = df["purity_H2"].values
H2_CO_ratio = df["H2_CO_ratio"].values
purity_CO2 = df["purity_CO2"].values
recovery_CO2 = df["recovery_CO2"].values
productivity = df["productivity"].values


model_name = "ARX"


In [2]:
# Colocando dados em desvio

y = np.array(
    [
        purity_H2,
        H2_CO_ratio,
        purity_CO2,
        recovery_CO2,
        productivity,
    ]
).T
u = np.array([t_feed, t_rinse, t_blow, t_purge]).T

# ciclo de referência
ref_idx = 1
y_ref = np.tile(y[ref_idx, :], (len(y), 1))
u_ref = np.tile(u[ref_idx, :], (len(u), 1))

# --- Calcular desvios ---
y_desvio = y - y_ref
u_desvio = u - u_ref


# Gerando modelos lineares


# ARX


In [3]:
from lib.utils import benchmark


def gerar_ARX(y, u):
    # Construindo a matriz de regressão
    X = np.hstack([y[:-1, :], u[:-1, :], u[1:, :]])

    # Saídas futuras
    Y = y[1:, :]

    # Estimando parâmetros via mínimos quadrados
    theta = np.linalg.lstsq(X, Y)[0]
    return theta


def inferir_ARX(theta, y, u):
    N = len(y)
    y_arx = np.zeros_like(y)
    y_arx[0] = y[0]  # inicializa com o primeiro valor real

    for k in range(1, N):
        phi = np.concatenate([y_arx[k - 1], u[k - 1], u[k]])
        y_arx[k] = phi @ theta
    return y_arx


theta = gerar_ARX(y_desvio, u_desvio)
print("Parâmetros estimados:\n", theta)

y_arx = inferir_ARX(theta, y_desvio, u_desvio)
y_arx = y_arx + y_ref  # Convertendo o valor desvio em variavel de engenharia

# Benchmarking
benchmark(inferir_ARX, model_name, theta, y_desvio, u_desvio)


Parâmetros estimados:
 [[ 6.94868674e-01  1.25705312e-03  6.91718599e-01  2.56492361e+00
   1.06464145e-01]
 [-5.78555812e+00  9.00180948e-01  1.60337045e+01  1.54957375e+01
   9.32061204e-01]
 [ 1.58362335e-01 -1.55207059e-02 -1.05463097e-01 -2.92663003e-01
  -1.70985405e-02]
 [-1.01645079e-01  6.62434505e-03  5.57311237e-02  1.62343444e-01
  -1.40050438e-03]
 [ 2.21273148e+00  5.06655863e-04  4.64759182e+00  1.47766797e+01
   8.59008578e-01]
 [-9.61508029e-03  1.08527222e-03  2.22477657e-02  4.32098216e-02
   1.84504456e-03]
 [-1.80919901e-04  3.36292341e-03  1.09472934e-01  2.52690387e-01
   1.21415322e-02]
 [ 9.66165431e-03 -3.58805266e-04  1.15199078e-02  4.62343511e-02
   1.81294621e-03]
 [ 4.26240273e-02 -2.93801679e-03 -1.18259467e-01 -3.44773705e-01
  -1.51825786e-02]
 [-7.51387395e-03 -5.97127020e-04  5.76405249e-03 -6.08635361e-02
   4.31612631e-04]
 [-1.93122399e-02 -1.87119611e-03  5.07010564e-02 -1.00126979e-01
  -7.96761147e-03]
 [-7.55188009e-04  3.12276012e-04  4.84230

In [4]:
# Plotando resultados
from lib.plot import plot_comparison

plot_comparison(ciclos, y, y_arx, model_name)


## Espaço de estados


In [5]:
model_name = "state-space"


def ARX_to_SS(theta):
    A = theta[:5, :].T
    B = theta[5:, :].T
    C = np.eye(5)
    D = np.zeros((5, 8))
    return A, B, C, D


A, B, C, D = ARX_to_SS(theta)


def inferir_SS(y, u):
    x = y_desvio[0].copy()  # inicializa com o primeiro valor real
    y_ss = np.zeros_like(y_desvio)
    y_ss[0] = x

    N = len(y)
    for k in range(1, N):
        u_ext = np.concatenate([u[k - 1], u[k]])
        x = A @ x + B @ u_ext
        y_ss[k] = C @ x + D @ u_ext
    return y_ss


y_ss = inferir_SS(y_desvio, u_desvio)
y_ss = y_ss + y_ref  # Convertendo o valor desvio em variavel de engenharia

# Benchmarking
benchmark(inferir_SS, model_name, y_desvio, u_desvio)


Benchmark file already exists. Skipping benchmark.


In [6]:
# Plotando resultados
plot_comparison(ciclos, y, y_ss, model_name)

print("A:", A)
print("B:", B)
print("C:", C)
print("D:", D)


A: [[ 6.94868674e-01 -5.78555812e+00  1.58362335e-01 -1.01645079e-01
   2.21273148e+00]
 [ 1.25705312e-03  9.00180948e-01 -1.55207059e-02  6.62434505e-03
   5.06655863e-04]
 [ 6.91718599e-01  1.60337045e+01 -1.05463097e-01  5.57311237e-02
   4.64759182e+00]
 [ 2.56492361e+00  1.54957375e+01 -2.92663003e-01  1.62343444e-01
   1.47766797e+01]
 [ 1.06464145e-01  9.32061204e-01 -1.70985405e-02 -1.40050438e-03
   8.59008578e-01]]
B: [[-9.61508029e-03 -1.80919901e-04  9.66165431e-03  4.26240273e-02
  -7.51387395e-03 -1.93122399e-02 -7.55188009e-04  3.92863548e-04]
 [ 1.08527222e-03  3.36292341e-03 -3.58805266e-04 -2.93801679e-03
  -5.97127020e-04 -1.87119611e-03  3.12276012e-04  7.73617910e-05]
 [ 2.22477657e-02  1.09472934e-01  1.15199078e-02 -1.18259467e-01
   5.76405249e-03  5.07010564e-02  4.84230257e-03 -7.30129390e-02]
 [ 4.32098216e-02  2.52690387e-01  4.62343511e-02 -3.44773705e-01
  -6.08635361e-02 -1.00126979e-01  1.04256548e-02  2.96887839e-01]
 [ 1.84504456e-03  1.21415322e-02  1